# Performance and Memory

When we use Python for automation, and if we're expecting to use the same code--or variations of it--over and over again, then it's worthwhile to figure out what's slowing it down. 

## Memory: Access and store only what you need, no more

tl;dr 
Look out for unnecessary memory use.
- Avoid lists where you could use tuples
- Avoid lists for checking membership where you could use sets
- Avoid sequences where you could use iterators

### lists vs. tuples

If you have a collection of values, your first thought may be to store them in a list.

In [37]:
data_list = [17999712, 2015, 'Hawkins Road', 'Linden ', 'NC', 28356]

Lists are nice because they are very flexible. You can change the values in the list, including appending and removing values. But that flexibility comes at a cost. Lists are less efficient than tuples. For example, they use more memory.

In [38]:
import sys

data_tuple = (17999712, 2015, 'Hawkins Road', 'Linden ', 'NC', 28356)

print(sys.getsizeof(data_list))
print(sys.getsizeof(data_tuple))

104
88


If you aren't going to be changing the values in a collection, use a tuple instead of a list.

### lists vs. sets

Lists perform faster for ordered operations (think: sorting). But sets and dictionaries are better for lookup.

When you want to see if an element already exists in a collection of elements, use a set to store that collection if possible. Lists and tuples require sequential lookup to see if an element is a member of the collection. That means that on average, it has to make n/2 comparisons for a collection of length n. Because every element in a set is unique, however, a set can use a hash table for lookups. That means no matter how big the collection is, the set only ever has to check 1 value. The bigger the collection is, the bigger the gap between sets and lists becomes. 
The hash is what makes sets (and dicts) so fast for lookups. Instead of searching every item, Python jumps directly to the hash bucket. When you add an element to a set, 1) Python calls the element’s __hash__() method to get a hash value (an integer); 2) That hash value determines where the element will be stored in the set's internal structure; 3) When checking if an element is in the set, Python uses the hash to quickly find it. 

The example below shows that a set is over 1000x faster than a list in calculating the first 100,000 values of [Recaman's sequence](https://oeis.org/search?q=recaman&language=english&go=Search) but the concept applies any time you are checking for membership in a large collection.

In [19]:
def recaman_check(cur, i, visited):
    return (cur - i) < 0 or (cur - i) in visited

def recaman_list(n: int) -> list[int]:
    """
    return a list of the first n numbers of the Recaman series
    """
    
    visited_list = [0]
    current = 0
    for i in range(1, n):
        if recaman_check(current, i, visited_list):
            current += i
        else:
            current -= i
        visited_list.append(current)
    return visited_list

In [20]:
%%timeit
recaman_list(100000)

19.2 s ± 20.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
def recaman_set(n: int) -> list[int]:
    visited_set = {0}
    current = 0
    for i in range(1, 100_000):
        if recaman_check(current, i, visited_set):
            current += i
        else:
            current -= i
        visited_set.add(current)
    return visited_set

In [22]:
%%timeit
recaman_set(100000)

15.1 ms ± 94.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### sequences vs. iterators

Regular functions and comprehensions typically create a container type like a list or a dictionary to store the data that results from the function’s intended computation. All this data is stored in memory at the same time.

In contrast, iterators keep only one data item in memory at a time, generating the next items on demand or lazily. 

With iterators and generators, you don’t need to store all the data in your compter’s memory at the same time.

Iterators and generators also allow you to completely decouple iteration from processing individual items. They let you connect multiple data processing stages to create memory-efficient data processing pipelines.

https://www.linkedin.com/pulse/python-generators-understanding-power-different-use-cases-singh/
https://nedbatchelder.com/text/iter.html
https://github.com/pythological/kanren
https://discuss.python.org/t/the-pros-cons-of-generators/30950

## Performance: Profile your code for bottlenecks

tl;dr Make time for performance checks anytime your code takes longer than ___ . There are several resources:
1. %%timeit is useful when...
Use the `%%timeit` notebook magic to perform some basic profiling. We know that the first cell is much slower than the second. But `%%timeit` isn't precise enough to tell which calls in each cell are taking the longest to execute.
2. cProfile is useful when...
The `cProfile` module has the ability to break down call by call to determine the number of calls and the total time spent on each. 
Look at tottime to find which functions are slow themselves. Look at cumtime to find functions that might call other slow functions.

## QA Workflow for Performance and Memory
1. Spot-check for instances of unnecessary memory use
1. Replace above instances with low-memory alternatives
1. If necessary, create a sample to profile on: same complexity, smaller size. Then:
1. Profile: Check for speed bottlenecks at a high level (%%timeit) 
1. Profile: For the slowest cell from prev step: check for speed bottlenecks at a granular level (cProfile)

---

# Exercise

__Exercise steps:__
1. Replace lists with memory-saving alternatives
    1. Replace with tuple
    1. Replace with set
    1. Replace with set (2nd example)
    1. Compare differences in speed with `%%timeit`
1. Replace sequences with memory-saving alternatives
    1. Convert list comprehension to generator expression
    1. Convert loop to intersection
    1. Compare differences in speed with `%%timeit`
1. Check for speed bottlenecks in detail with `cProfile`
    1. Apply for all the above code blocks
    1. Inspect results
1. Challenge: Improve readability across the script

## 0) Prepare workspace

In [ ]:
import csv

## 1) Replace lists with memory-saving alternatives

Scenario: You have CSVs stored in a subdirectory. Some of these CSVs can be converted to point shapefiles. For every CSV in the folder, you need to determine whether it contains a field called 'lat', which would indicate it has point coordinates.

In [ ]:
# Create list of file paths where the data is stored.
myDataPaths = [filePath for file in directory]


# Define a function for determining if a file meets your criteria.
def meetsCriteria(filePaths):
    """
    Dataframe must have a 'lat' field to be included.
    """
    members = []
    criterium = 'lat'
    
    for filePath in filePaths:
        with open(filePath) as fPath:
            headerList = csv.DictReader(fPath).fieldnames
            if criterium in headerList:
                members.append(filePath)
    return members


# Print all matching file paths
print(meetsCriteria(myDataPaths))

### Change a list to a tuple to reduce its storage.

Change the filePaths list to a tuple.

In [ ]:
# Exercise solution
myDataPaths = (filePath for file in directory)

### Identify membership using set intersection instead of list look-up

For the *meetsCriteria()* function, search for each CSV's fieldnames using set intersection.
<br><br>Actions:
- Change the if statement to use *.intersection()*

In [2]:
# Exercise solution
def meetsCriteria(filePaths):
    """
    Dataframe must have a 'lat' field to be included.
    """
    members = []
    criterium = 'lat'
    
    for filePath in filePaths:
        with open(filePath) as fPath:
            headerList = csv.DictReader(fPath).fieldnames
            if headerList.intersection(criterium) is not None:
                members.append(filePath)
    return members

### Change list to a set to reduce its storage.

Adapt the meetsCriteria function to add files to a set instead of appending to a list.
<br><br>Actions:
- Change the variable *members* to a set.
- Modify the line *members.append(filePath)* to use the *.add()* function.

In [ ]:
# Exercise solution
def meetsCriteria(filePaths):
    """
    Dataframe must have a 'lat' field to be included.
    """
    members = {}
    criterium = 'lat'
    
    for filePath in filePaths:
        with open(filePath) as fPath:
            headerList = csv.DictReader(fPath).fieldnames
            if headerList.intersection(criterium) is not None:
                members.add(filePath)
    return members

### Compare differences in speed using `%%timeit`

Using `%%timeit`, compare the time it took to create myDataPaths as a list (original code) versus as a tuple (exercise solution).

In [ ]:
%%timeit
print([filePath for file in directory])

In [ ]:
%%timeit
## Your solution here ##

Use `%%timeit` again to compare list-based lookup to set intersection.

In [ ]:
%%timeit
def meetsCriteria(filePaths):
    """
    Dataframe must have a 'lat' field to be included.
    """
    members = []
    criterium = 'lat'
    
    for filePath in filePaths:
        with open(filePath) as fPath:
            headerList = csv.DictReader(fPath).fieldnames
            if criterium in headerList:
                members.append(filePath)
    return members


# Print all matching file paths
print(meetsCriteria(myDataPaths))

In [ ]:
%%timeit
## Your solution here ##

Finally, compare the second list vs. set change that you made. 

In [ ]:
%%timeit
def meetsCriteria(filePaths):
    """
    Dataframe must have a 'lat' field to be included.
    """
    members = []
    criterium = 'lat'
    
    for filePath in filePaths:
        with open(filePath) as fPath:
            headerList = csv.DictReader(fPath).fieldnames
            if criterium in headerList:
                members.append(filePath)
    return members


# Print all matching file paths
print(meetsCriteria(myDataPaths))

In [ ]:
%%timeit
## Your solution here ##

## 2) Replace sequences with memory-saving alternatives

## 3) Check for speed bottlenecks in detail

### Use cProfile to locate the slowest calls in the original script.

### Use cProfile to locate the slowest calls in your improved script. 
Hint: Sort by tottime instead of name to find hotspots more easily.

Solution from the previous steps:

In [ ]:
# Create list of file paths where the data is stored.
myDataPaths = (filePath for file in directory)


# Define a function for determining if a file meets your criteria.
def meetsCriteria(filePaths):
    """
    Dataframe must have a 'lat' field to be included.
    """
    members = {}
    criterium = 'lat'
    
    for filePath in filePaths:
        with open(filePath) as fPath:
            headerList = csv.DictReader(fPath).fieldnames
            if headerList.intersection(criterium) is not None:
                members.add(filePath)
    return members


# Print all matching file paths
print(meetsCriteria(myDataPaths))

## 4) Challenge: Improve readability across the script

For the function *meetsCriteria()*, add more detail in the doc string.

In [ ]:
## Your solution here ##

# Notes

Imagine you have a database (geopackage) of a city's building footprint geometries. Each layer represents the new construction built in that year. What you need instead, however, is a snapshot of total built area each year: all of that year's construction plus anything existing from previous years in the study.

To create this new database, for every year in the study, you will subset building features for that year and all previous years, dissolve them to create one cumulative feature, and then save to file.

In [ ]:
ftPrints = gpd.read_file("City.gpkg", layer="Footprints")
constructYears = [1999, 2000, 2001, 2002, 2003, 2004]
startYear = 1999
print('Prepared to dissolve building footprints for cumulative years from 1999 to 2004.')

#### BEFORE: for loop

One way to perform this workflow is with a for loop:

In [ ]:
for year in constructYears:
    timeframe = ftPrints[ftPrints['year'].between(startYear, year, inclusive=True)] 
    tfDissolve = timeframe.dissolve(by='ID',
                                      aggfunc={"year": "max"},
                                      as_index=False)
    
    yearName = ''.join(['cu', str(year)])
    tfDissolve.to_file(driver='GPKG', filename='cuFootprints.gpkg', layer=yearName)

#### AFTER: generator
*(exercise solution)*

You can improve this code by creating a data pipeline of generator expressions, ending in a for loop for only the final step.

In [ ]:
# First generator
timeframes = (ftPrints[ftPrints['year'].between(startYear, year, inclusive=True)] for year in constructYears)

# Second generator
tfDissolve = (timeframe.dissolve(by='ID', aggfunc={"year": "max"}, as_index=False) for timeframe in timeframes)

# Third generator
yearName = (''.join(['cu', str(year)]) for year in constructYears)

# Commence iteration
for year in constructYears:
    tfDissolve.to_file(driver='GPKG', filename='cuFootprints.gpkg', layer=yearName)

## Profile your code instead of guessing about performance bottlenecks

In [19]:
def recaman_check(cur, i, visited):
    return (cur - i) < 0 or (cur - i) in visited

def recaman_list(n: int) -> list[int]:
    """
    return a list of the first n numbers of the Recaman series
    """
    
    visited_list = [0]
    current = 0
    for i in range(1, n):
        if recaman_check(current, i, visited_list):
            current += i
        else:
            current -= i
        visited_list.append(current)
    return visited_list

In [20]:
%%timeit
recaman_list(100000)

19.2 s ± 20.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
def recaman_set(n: int) -> list[int]:
    visited_set = {0}
    current = 0
    for i in range(1, 100_000):
        if recaman_check(current, i, visited_set):
            current += i
        else:
            current -= i
        visited_set.add(current)
    return visited_set

In [22]:
%%timeit
recaman_set(100000)

15.1 ms ± 94.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The example above uses the `%%timeit` notebook magic to perform some basic profiling. We know that the first cell is much slower than the second. But `%%timeit` isn't precise enough to tell which calls in each cell are taking the longest to execute. Are sets faster because it's easier to check if an element is in a set? Or are sets faster because it is easier to add an element to a set than append an element to a list?

The `cProfile` module has the ability to break down call by call to determine the number of calls and the total time spent on each. Running each function with cProfile demonstrates that the in the list version of the function, the time to append elements is dwarfed by the time it takes to check if an element is in a list. The set version of the function is actually a little slower to add elements, but it is much faster to check elements for membership.

In [27]:
import cProfile

cProfile.run('recaman_list(100000)')

         200002 function calls in 19.888 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    99999   19.808    0.000   19.808    0.000 1555087545.py:1(recaman_check)
        1    0.072    0.072   19.888   19.888 1555087545.py:4(recaman_list)
        1    0.001    0.001   19.888   19.888 <string>:1(<module>)
        1    0.000    0.000   19.888   19.888 {built-in method builtins.exec}
    99999    0.007    0.000    0.007    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [26]:
cProfile.run('recaman_set(100000)')

         200002 function calls in 0.068 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    99999    0.018    0.000    0.018    0.000 1555087545.py:1(recaman_check)
        1    0.041    0.041    0.068    0.068 662776923.py:1(recaman_set)
        1    0.001    0.001    0.068    0.068 <string>:1(<module>)
        1    0.000    0.000    0.068    0.068 {built-in method builtins.exec}
    99999    0.009    0.000    0.009    0.000 {method 'add' of 'set' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




## Use more tuples and fewer lists
If you have a collection of values, your first thought may be to store them in a list.

In [ ]:
data_list = [17999712, 2015, 'Hawkins Road', 'Linden ', 'NC', 28356]

Lists are nice because they are very flexible. You can change the values in the list, including appending and removing values. But that flexibility comes at a cost. Lists are less efficient than tuples. For example, they use more memory.

In [ ]:
import sys

data_tuple = (17999712, 2015, 'Hawkins Road', 'Linden ', 'NC', 28356)

print(sys.getsizeof(data_list))
print(sys.getsizeof(data_tuple))

If you aren't going to be changing the values in a collection, use a tuple instead of a list.

## Use more iterators and fewer lists/tuples for loops

Regardless of whether you use a list or a tuple, you need to store a reference to every value in memory. Both lists and tuples take up more memory the bigger they get.

In [39]:
little_tuple = tuple(range(10))
big_tuple = tuple(range(10000))

print(sys.getsizeof(little_tuple))
print(sys.getsizeof(big_tuple))

120
80040


Lists and tuples are iterables because you can iterate over their values. Iterators are a special type of iterable that evaluate their values lazily, only when the values are needed. They don't hold references to all the values in memory. That means iterators have constant size in memory. For example, `enumerate` produces an iterator. ArcPy cursors also produce iterators (not shown here so that this notebook works in an standard Notebook Server environment without access to ArcPy).

In [40]:
from collections.abc import Iterator

little_tuple_iterator = enumerate(little_tuple)
big_tuple_iterator = enumerate(big_tuple)

print(isinstance(little_tuple_iterator, Iterator))
print(sys.getsizeof(little_tuple_iterator))
print(sys.getsizeof(big_tuple_iterator))

True
72
72


An iterator is any object that implements both the `__iter__` and `__next__` methods. You can make your own iterators by creating a custom class that implements those methods (or a class that inherits from `collections.abc.Iterator`). An easier way to create your own iterators is to create a special type of iterator called a generator. Generators are functions that use a `yield` statement to produce values lazily, one at a time (instead of all at once like a list or tuple). For example, you could use a generator to produce infinite sequential ObjectID numbers.

In [41]:
from types import GeneratorType

def make_oids(start):
    while True:
        yield start
        start += 1

oid = make_oids(start=1000)

is_generator = isinstance(oid, GeneratorType)
is_iterator = isinstance(oid, Iterator)
print(f"Is oid a generator? {is_generator}")
print(f"Is oid an iterator? {is_iterator}")

# Every time you want a new oid, just call next(oid). The size of oid never changes
print(f"At first the size of oid is {sys.getsizeof(oid)} bytes")
print(f"The first value produced by oid is {next(oid)}")

for i in range(10000):
    new_oid = next(oid)

print(f"After 10,000 iterations, the next value produced by oid is {next(oid)}")
print(f"After 10,000 iterations, the size of oid is {sys.getsizeof(oid)} bytes")

Is oid a generator? True
Is oid an iterator? True
At first the size of oid is 192 bytes
The first value produced by oid is 1000
After 10,000 iterations, the next value produced by oid is 11001
After 10,000 iterations, the size of oid is 192 bytes


Iterators (inlcuding generators) are useful for enabling better separation of concerns. Maybe you need to create many records and assign each of them an ObjectID. You could do that in a loop.

In [42]:
records = []
oid = 1000
for value in range(10):
    record = {
        'OID':  oid,
        'value': value,
    }
    oid += 1
    records.append(record)
records

[{'OID': 1000, 'value': 0},
 {'OID': 1001, 'value': 1},
 {'OID': 1002, 'value': 2},
 {'OID': 1003, 'value': 3},
 {'OID': 1004, 'value': 4},
 {'OID': 1005, 'value': 5},
 {'OID': 1006, 'value': 6},
 {'OID': 1007, 'value': 7},
 {'OID': 1008, 'value': 8},
 {'OID': 1009, 'value': 9}]

That works, but the loop is doing two things: making a record and making an OID value. It would be better if those things were separated. That way if we later need to change the way OID values are generated, we don't have to change the code in the loop. 

In [43]:
records = []
oid = make_oids(1000)
for value in range(10):
    record = {
        'OID':  next(oid),
        'value': value,
    }
    records.append(record)
records

[{'OID': 1000, 'value': 0},
 {'OID': 1001, 'value': 1},
 {'OID': 1002, 'value': 2},
 {'OID': 1003, 'value': 3},
 {'OID': 1004, 'value': 4},
 {'OID': 1005, 'value': 5},
 {'OID': 1006, 'value': 6},
 {'OID': 1007, 'value': 7},
 {'OID': 1008, 'value': 8},
 {'OID': 1009, 'value': 9}]